# calss
# 1. common class

In [300]:
# vocab loading
import sentencepiece as spm

VOCAB_PATH = "/home/henry/Documents/wrapper/source"
vocab_file = f"{VOCAB_PATH}/kowiki.model"
vocab = spm.SentencePieceProcessor()
vocab.load(vocab_file)

True

In [319]:
from types import SimpleNamespace
import torch.nn as nn
import torch.nn.functional as F
config = dict({
    "n_enc_vocab": len(vocab),
    "n_dec_vocab": len(vocab),
    "n_enc_seq": 256,
    "n_dec_seq": 256,
    "n_layer": 6,
    "d_hidn": 256,
    "i_pad": 0,
    "d_ff": 1024,
    "n_head": 4,
    "d_head": 64,
    "dropout": 0.1,
    "layer_norm_epsilon": 1e-12,
    "n_output": 2,
    "device": device
})
print(config)
config = SimpleNamespace(**config)

{'n_enc_vocab': 8007, 'n_dec_vocab': 8007, 'n_enc_seq': 256, 'n_dec_seq': 256, 'n_layer': 6, 'd_hidn': 256, 'i_pad': 0, 'd_ff': 1024, 'n_head': 4, 'd_head': 64, 'dropout': 0.1, 'layer_norm_epsilon': 1e-12, 'n_output': 2}


In [302]:
def get_sinusoid_encoding_table(n_seq, d_hidn):
    def cal_angle(position, i_hidn):
        return position / np.power(10000, 2 * (i_hidn // 2) / d_hidn)
    def get_posi_angle_vec(position):
        return [cal_angle(position, i_hidn) for i_hidn in range(d_hidn)]

    sinusoid_table = np.array([get_posi_angle_vec(i_seq) for i_seq in range(n_seq)])
    sinusoid_table[:, 0::2] = np.sin(sinusoid_table[:, 0::2])  # even index sin 
    sinusoid_table[:, 1::2] = np.cos(sinusoid_table[:, 1::2])  # odd index cos

    return sinusoid_table

def get_attn_pad_mask(seq_q, seq_k, i_pad):
    """
    key_vector의 pad열은 모두 0으로 padding
    - row: query token
    - col: key token
    
    params
    - seq_q, seq_k: [bs, len_seq]
    """
    batch_size, len_q = seq_q.size()
    batch_size, len_k = seq_k.size()
    pad_attn_mask = seq_k.data.eq(i_pad).unsqueeze(1).expand(batch_size, len_q, len_k)  # key vector에 masking한 것을, len_q만큼 늘려줌(row wise)
    return pad_attn_mask

def get_attn_decoder_mask(seq):
    subsequent_mask = torch.ones_like(seq).unsqueeze(-1).expand(seq.size(0), seq.size(1), seq.size(1))
    subsequent_mask = subsequent_mask.triu(diagonal=1) # upper triangular part of a matrix
    return subsequent_mask

### attn pad mask example

In [38]:
sample_key_seq = torch.randint(0,3, size=(2, 4)) # [bs, q_seq_len]
sample_key_seq

tensor([[0, 2, 2, 1],
        [2, 2, 2, 0]])

In [39]:
sample_key_seq.unsqueeze(1) # [2,4] -> [2, 1, 4]

tensor([[[0, 2, 2, 1]],

        [[2, 2, 2, 0]]])

In [40]:
sample_key_seq.unsqueeze(1).expand(2, 4, 4)

tensor([[[0, 2, 2, 1],
         [0, 2, 2, 1],
         [0, 2, 2, 1],
         [0, 2, 2, 1]],

        [[2, 2, 2, 0],
         [2, 2, 2, 0],
         [2, 2, 2, 0],
         [2, 2, 2, 0]]])

In [41]:
attn_pad = sample_key_seq.eq(0).unsqueeze(1).expand(2, 4, 4) # attn_pad
attn_pad

tensor([[[ True, False, False, False],
         [ True, False, False, False],
         [ True, False, False, False],
         [ True, False, False, False]],

        [[False, False, False,  True],
         [False, False, False,  True],
         [False, False, False,  True],
         [False, False, False,  True]]])

### get_attn_decoder_mask example

In [49]:
decoder_mask = torch.ones_like(sample_key_seq).unsqueeze(-2).expand(sample_key_seq.size(0), sample_key_seq.size(1), sample_key_seq.size(1))
decoder_mask

tensor([[[1, 1, 1, 1],
         [1, 1, 1, 1],
         [1, 1, 1, 1],
         [1, 1, 1, 1]],

        [[1, 1, 1, 1],
         [1, 1, 1, 1],
         [1, 1, 1, 1],
         [1, 1, 1, 1]]])

In [57]:
decoder_mask.triu(1) # convert upper diagonal part into 1

tensor([[[0, 1, 1, 1],
         [0, 0, 1, 1],
         [0, 0, 0, 1],
         [0, 0, 0, 0]],

        [[0, 1, 1, 1],
         [0, 0, 1, 1],
         [0, 0, 0, 1],
         [0, 0, 0, 0]]])

### decoder part masking example
-  logical operation

In [80]:
attn_pad

tensor([[[ True, False, False, False],
         [ True, False, False, False],
         [ True, False, False, False],
         [ True, False, False, False]],

        [[False, False, False,  True],
         [False, False, False,  True],
         [False, False, False,  True],
         [False, False, False,  True]]])

In [81]:
decoder_mask.triu(1)

tensor([[[False,  True,  True,  True],
         [False, False,  True,  True],
         [False, False, False,  True],
         [False, False, False, False]],

        [[False,  True,  True,  True],
         [False, False,  True,  True],
         [False, False, False,  True],
         [False, False, False, False]]])

In [71]:
# 'or' opeartion
# True+False==True; True+True==True; False+False==False; <- or(mask==True)==True
attn_pad + decoder_mask.triu(1) 

tensor([[[ True,  True,  True,  True],
         [ True, False,  True,  True],
         [ True, False, False,  True],
         [ True, False, False, False]],

        [[False,  True,  True,  True],
         [False, False,  True,  True],
         [False, False, False,  True],
         [False, False, False,  True]]])

In [82]:
decoder_mask = torch.gt(attn_pad + decoder_mask.triu(1), 0) # gt, not eq: mask the part of value, true 
decoder_mask

tensor([[[ True,  True,  True,  True],
         [ True, False,  True,  True],
         [ True, False, False,  True],
         [ True, False, False, False]],

        [[False,  True,  True,  True],
         [False, False,  True,  True],
         [False, False, False,  True],
         [False, False, False,  True]]])

In [303]:
class PoswiseFeedForwardNet(nn.Module):
    """
    variables
    - inputs: output of attn layer(attn_outputs)
        - shape:[bs, len_query_seq, d_hidn]
    return
    - output
    """
    def __init__(self, config):
        super().__init__()
        self.config = config

        self.conv1 = nn.Conv1d(in_channels=self.config.d_hidn, out_channels=self.config.d_ff, kernel_size=1)
        self.conv2 = nn.Conv1d(in_channels=self.config.d_ff, out_channels=self.config.d_hidn, kernel_size=1)
        self.active = nn.gelu()
        self.dropout = nn.Dropout(config.dropout)

    def forward(self, inputs):
        # (bs, d_ff, n_seq)
        output = self.active(self.conv1(inputs.transpose(1, 2)))
        # (bs, n_seq, d_hidn)
        output = self.conv2(output).transpose(1, 2)
        output = self.dropout(output)
        # (bs, n_seq, d_hidn)
        return output

### PoswiseFeedForwardNet example code

In [130]:
# multihead output: [bs, seq_len, d_hidn]
# how to apply 1d conv to NLP in torch: https://gist.github.com/spro/c87cc706625b8a54e604fb1024106556
attn_outputs = torch.randint(1, 10, size=[2, 4, 8]).float()
attn_outputs

tensor([[[1., 9., 3., 7., 9., 3., 2., 4.],
         [4., 4., 1., 2., 4., 5., 1., 9.],
         [5., 9., 4., 8., 9., 5., 4., 5.],
         [5., 7., 2., 3., 5., 2., 8., 2.]],

        [[1., 1., 9., 6., 1., 2., 9., 9.],
         [4., 3., 8., 7., 5., 3., 1., 1.],
         [5., 6., 7., 3., 5., 8., 7., 1.],
         [8., 8., 8., 3., 3., 9., 2., 4.]]])

In [151]:
# input dim [bs, d_hidn, seq_len]
conv1d_1st = nn.Conv1d(in_channels=8, out_channels=128, kernel_size=1)(attn_outputs.transpose(2, 1))
conv1d_1st.shape # [bs, out_dim, seq_len]

torch.Size([2, 128, 4])

In [155]:
conv1d_2nd = nn.Conv1d(in_channels=128, out_channels=8, kernel_size=1)(conv1d_1st)
conv1d_2nd.shape # [bs, d_hidn, seq_len]

torch.Size([2, 8, 4])

In [159]:
conv1d_2nd.transpose(2,1).shape # [bs, seq_len, d_hidn]

torch.Size([2, 4, 8])

In [304]:
class ScaledDotProductAttention(nn.Module):
    """
    variables
    - V, Q: [bs, len_seq, d_hidn]
    - attn_prob: [bs, n_head, len_query_seq, len_key_seq]
    - context: [bs, n_head, len_query_seq, d_hidn]
    calculate
    - scores: Q * K.T = [bs, len_seq, d_hidn] * [bs, d_hidn, len_seq]
        - shape: [bs, len_seq, len_seq]
    - attn_prob: dropout(softmax(score))
    - context: torch.matmul(attn_prob, Value vector)
    """
    def __init__(self, config):
        super().__init__()
        self.config = config
        self.dropout = nn.Dropout(config.dropout)
        self.scale = 1 / (self.config.d_head ** 0.5)
    
    def forward(self, Q, K, V, attn_mask):
        scores = torch.matmul(Q, K.transpose(-1, -2)).mul_(self.scale)
        scores.masked_fill_(attn_mask, -1e9) # softmax에서 sum!=0 위해 -1e9
        
        attn_prob = nn.Softmax(dim=-1)(scores)
        attn_prob = self.dropout(attn_prob)
        
        context = torch.matmul(attn_prob, V)
        # (bs, n_head, n_q_seq, d_v), (bs, n_head, n_q_seq, n_v_seq)
        return context, attn_prob

### scaled dot example

In [95]:
attn_pad # [bs, q_seq_len, k_seq_len]

tensor([[[False, False, False,  True],
         [False, False, False,  True],
         [False, False, False,  True],
         [False, False, False,  True]],

        [[False, False, False,  True],
         [False, False, False,  True],
         [False, False, False,  True],
         [False, False, False,  True]]])

In [98]:
scores = torch.randint_like(attn_pad, 2, 10, dtype=torch.float)
scores

tensor([[[4., 8., 6., 5.],
         [2., 9., 7., 2.],
         [5., 9., 9., 8.],
         [9., 9., 4., 4.]],

        [[9., 9., 8., 7.],
         [4., 8., 6., 8.],
         [8., 3., 2., 7.],
         [6., 8., 7., 5.]]])

In [99]:
scores.masked_fill_(attn_pad, -1e9)

tensor([[[ 4.0000e+00,  8.0000e+00,  6.0000e+00, -1.0000e+09],
         [ 2.0000e+00,  9.0000e+00,  7.0000e+00, -1.0000e+09],
         [ 5.0000e+00,  9.0000e+00,  9.0000e+00, -1.0000e+09],
         [ 9.0000e+00,  9.0000e+00,  4.0000e+00, -1.0000e+09]],

        [[ 9.0000e+00,  9.0000e+00,  8.0000e+00, -1.0000e+09],
         [ 4.0000e+00,  8.0000e+00,  6.0000e+00, -1.0000e+09],
         [ 8.0000e+00,  3.0000e+00,  2.0000e+00, -1.0000e+09],
         [ 6.0000e+00,  8.0000e+00,  7.0000e+00, -1.0000e+09]]])

In [101]:
attn_prob = nn.Softmax(dim=-1)(scores.float())
attn_prob

tensor([[[1.5876e-02, 8.6681e-01, 1.1731e-01, 0.0000e+00],
         [8.0254e-04, 8.8009e-01, 1.1911e-01, 0.0000e+00],
         [9.0747e-03, 4.9546e-01, 4.9546e-01, 0.0000e+00],
         [4.9832e-01, 4.9832e-01, 3.3577e-03, 0.0000e+00]],

        [[4.2232e-01, 4.2232e-01, 1.5536e-01, 0.0000e+00],
         [1.5876e-02, 8.6681e-01, 1.1731e-01, 0.0000e+00],
         [9.9087e-01, 6.6764e-03, 2.4561e-03, 0.0000e+00],
         [9.0031e-02, 6.6524e-01, 2.4473e-01, 0.0000e+00]]])

In [108]:
V = torch.randint(2, 10, size=(2,4,4)).float()
V

tensor([[[2., 5., 9., 6.],
         [2., 9., 5., 5.],
         [5., 4., 6., 5.],
         [8., 2., 7., 4.]],

        [[4., 9., 6., 8.],
         [7., 6., 4., 6.],
         [6., 4., 3., 3.],
         [6., 5., 6., 7.]]])

In [119]:
nn.Dropout()(attn_prob)

tensor([[[0.0000, 1.7336, 0.2346, 0.0000],
         [0.0000, 1.7602, 0.0000, 0.0000],
         [0.0181, 0.9909, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0067, 0.0000]],

        [[0.8446, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [1.9817, 0.0134, 0.0000, 0.0000],
         [0.0000, 1.3305, 0.4895, 0.0000]]])

In [120]:
context = torch.matmul(nn.Dropout()(attn_prob), V)
context

tensor([[[ 0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000],
         [ 6.9365, 12.8820, 10.9002,  9.9093],
         [ 1.9933,  4.9832,  8.9698,  5.9799]],

        [[ 5.2429,  8.8446,  6.0000,  7.6893],
         [ 1.5347,  1.2243,  0.8944,  0.9579],
         [ 0.0000,  0.0000,  0.0000,  0.0000],
         [ 9.3134,  7.9829,  5.3219,  7.9829]]])

In [305]:
class MultiHeadAttention(nn.Module):
    """
    weight mat
    - W_Q, W_K, W_K: [d_hidn, n_head*d_head]
    
    vectors
    - q_s, k_s, v_s: [bs, n_head, seq_len, d_head]
    
    calculate
    - eg. W_Q(Q): Q * W_Q = [bs, seq_len, d_hidn] * [d_hidn, n_head*d_head] 
                            = [bs, seq_len, n_head*d_head] 
                            <- ([seq_len * d_head] mat = embedded Q가 n_head만큼 있음) * bs
    - eg. W_Q(Q).view(bs, -1, n_head, d_head).transpose(1, 2) = (bs, n_head, -1=seq_len, d_head) 
    
    return
    - output: [bs, len_query_seq, d_hidn]
    - attn_prob: [bs, n_head, len_query_seq, len_key_seq]
    """
    def __init__(self, config):
        super().__init__()
        self.config = config
        
        # n_head의 K, Q, V 한번에 생성
        self.W_Q = nn.Linear(self.config.d_hidn, self.config.n_head * self.config.d_head)
        self.W_K = nn.Linear(self.config.d_hidn, self.config.n_head * self.config.d_head)
        self.W_V = nn.Linear(self.config.d_hidn, self.config.n_head * self.config.d_head)
        self.scaled_dot_attn = ScaledDotProductAttention(self.config)
        self.linear = nn.Linear(self.config.n_head * self.config.d_head, self.config.d_hidn)
        self.dropout = nn.Dropout(config.dropout)
    # n_head의 K, Q, V 한번에 생성
    def forward(self, Q, K, V, attn_mask):
        batch_size = Q.size(0)
        q_s = self.W_Q(Q).view(batch_size, -1, self.config.n_head, self.config.d_head).transpose(1,2)
        k_s = self.W_K(K).view(batch_size, -1, self.config.n_head, self.config.d_head).transpose(1,2)
        v_s = self.W_V(V).view(batch_size, -1, self.config.n_head, self.config.d_head).transpose(1,2)

        attn_mask = attn_mask.unsqueeze(1).repeat(1, self.config.n_head, 1, 1)

        context, attn_prob = self.scaled_dot_attn(q_s, k_s, v_s, attn_mask)
        context = context.transpose(1, 2).contiguous().view(batch_size, -1, self.config.n_head * self.config.d_head) # [bs, seq_len, n_head * d_head]
        output = self.linear(context) # 각 head의 output은 position wise sum이므로 해당 calculate 가능
        output = self.dropout(output)
        
        return output, attn_prob

### multihead forward part example
- eg.1

In [ ]:
# [bs, len_seq, embed_dim=d_hidn]
input_vector.shape

In [54]:
Q = input_vector # [bs, seq_len, d_hidn]
Qs = nn.Linear(in_features=128, out_features=8*128)(Q)
Qs.shape # [bs, seq_len, n_head*d_hidn]

torch.Size([2, 8, 1024])

- eg.2: context vector 변환
    - reshape & transpose했을 때 `[19, 23,  7, 92, 66, 10, 15, 73, 19,  5, 81, 80, 49, 92, 79]` 변화


In [70]:
A = np.random.randint(1, 100, size=(2,4,3*5)) # [bs, seq_len, n_head * d_head]
A

array([[[19, 23,  7, 92, 66, 10, 15, 73, 19,  5, 81, 80, 49, 92, 79],
        [29, 78, 13, 54, 29, 49, 56, 55,  5, 47, 87, 92, 34, 18, 75],
        [72, 68, 10, 57, 34, 47, 46, 63,  3, 49, 40, 87, 79, 92, 41],
        [ 5, 96, 76, 14, 73, 64, 90, 41, 69, 18, 95, 75, 96, 72, 30]],

       [[91, 42, 91, 98,  1, 17, 64, 33, 35, 26,  2, 94, 33, 23, 53],
        [43, 61, 49, 26, 86, 10, 61, 90, 89, 24, 68, 21, 28, 12, 64],
        [57, 68, 77, 74, 10,  3, 99, 29, 81, 97, 24, 65, 63, 91, 73],
        [93,  1, 47, 60, 90, 49, 21, 90, 96, 41, 29, 53, 59, 57, 42]]])

In [73]:
A.reshape(2,-1,3,5) # [bs, seq_len, n_head, d_head]

array([[[[19, 23,  7, 92, 66],
         [10, 15, 73, 19,  5],
         [81, 80, 49, 92, 79]],

        [[29, 78, 13, 54, 29],
         [49, 56, 55,  5, 47],
         [87, 92, 34, 18, 75]],

        [[72, 68, 10, 57, 34],
         [47, 46, 63,  3, 49],
         [40, 87, 79, 92, 41]],

        [[ 5, 96, 76, 14, 73],
         [64, 90, 41, 69, 18],
         [95, 75, 96, 72, 30]]],


       [[[91, 42, 91, 98,  1],
         [17, 64, 33, 35, 26],
         [ 2, 94, 33, 23, 53]],

        [[43, 61, 49, 26, 86],
         [10, 61, 90, 89, 24],
         [68, 21, 28, 12, 64]],

        [[57, 68, 77, 74, 10],
         [ 3, 99, 29, 81, 97],
         [24, 65, 63, 91, 73]],

        [[93,  1, 47, 60, 90],
         [49, 21, 90, 96, 41],
         [29, 53, 59, 57, 42]]]])

In [78]:
np.transpose(A.reshape(2,-1,3,5), axes=[0,2,1,3]) # [bs, n_head, seq_len, d_head]

array([[[[19, 23,  7, 92, 66],
         [29, 78, 13, 54, 29],
         [72, 68, 10, 57, 34],
         [ 5, 96, 76, 14, 73]],

        [[10, 15, 73, 19,  5],
         [49, 56, 55,  5, 47],
         [47, 46, 63,  3, 49],
         [64, 90, 41, 69, 18]],

        [[81, 80, 49, 92, 79],
         [87, 92, 34, 18, 75],
         [40, 87, 79, 92, 41],
         [95, 75, 96, 72, 30]]],


       [[[91, 42, 91, 98,  1],
         [43, 61, 49, 26, 86],
         [57, 68, 77, 74, 10],
         [93,  1, 47, 60, 90]],

        [[17, 64, 33, 35, 26],
         [10, 61, 90, 89, 24],
         [ 3, 99, 29, 81, 97],
         [49, 21, 90, 96, 41]],

        [[ 2, 94, 33, 23, 53],
         [68, 21, 28, 12, 64],
         [24, 65, 63, 91, 73],
         [29, 53, 59, 57, 42]]]])

# 2. encoder clasas

In [355]:
class EncoderLayer(nn.Module):
    """
    variables
    - attn_outputs: [bs, lne_query_seq, d_hidn]
    - ffn_outputs: [bs, lne_query_seq, d_hidn]
    """
    def __init__(self, config):
        super().__init__()
        self.config = config

        self.self_attn = MultiHeadAttention(self.config)
        self.layer_norm1 = nn.LayerNorm(self.config.d_hidn, eps=self.config.layer_norm_epsilon)
        self.pos_ffn = PoswiseFeedForwardNet(self.config)
        self.layer_norm2 = nn.LayerNorm(self.config.d_hidn, eps=self.config.layer_norm_epsilon)
    
    def forward(self, inputs, attn_mask):
        att_outputs, attn_prob = self.self_attn(inputs, inputs, inputs, attn_mask)
        att_outputs = self.layer_norm1(inputs + att_outputs)
        
        ffn_outputs = self.pos_ffn(att_outputs)
        ffn_outputs = self.layer_norm2(ffn_outputs + att_outputs)
        
        return ffn_outputs, attn_prob

In [362]:
class Encoder(nn.Module):
    """
    variable
    - inputs: [bs, seq_len] <- inputs before embedding elems of which refer to voc_idx
    - outputs: embeded output; [bs, seq_len, d_hidn];
    - final outputs: result context vector of attn layers; [bs, seq_len, d_hidn];
    - attn_mask: [bs, seq_q_len, seq_k_len]
    - attn_prob: [bs, seq_q_len, d_hidn] <- 최종 결과물
    """
    def __init__(self, config):
        super().__init__()
        self.config = config

        self.enc_emb = nn.Embedding(self.config.n_enc_vocab, self.config.d_hidn)
        sinusoid_table = torch.FloatTensor(get_sinusoid_encoding_table(self.config.n_enc_seq + 1, self.config.d_hidn))
        self.pos_emb = nn.Embedding.from_pretrained(sinusoid_table, freeze=True)

        self.layers = nn.ModuleList([EncoderLayer(self.config) for _ in range(self.config.n_layer)])
    
    def forward(self, inputs):
        position = torch.arange(inputs.size(1), device=inputs.device, dtype=inputs.dtype).expand(inputs.size(0), inputs.size(1)).contiguous() + 1
        pos_mask = inputs.eq(self.config.i_pad)
        position.masked_fill_(pos_mask, 0)

        outputs = self.enc_emb(inputs) + self.pos_emb(position)
        
        attn_mask = get_attn_pad_mask(inputs, inputs, self.config.i_pad)

        attn_probs = []
        for layer in self.layers:
            outputs, attn_prob = layer(outputs, attn_mask) # outputs = ffn_outputs
            attn_probs.append(attn_prob)
        return outputs, attn_probs

### position example

In [34]:
inputs = torch.randint(1,10,size=(2, 8))
inputs

tensor([[4, 8, 4, 6, 4, 9, 3, 2],
        [5, 1, 2, 1, 2, 1, 8, 2]])

In [31]:
position = torch.arange(inputs.size(1)).expand(2, 8).contiguous()+1
position

tensor([[1, 2, 3, 4, 5, 6, 7, 8],
        [1, 2, 3, 4, 5, 6, 7, 8]])

In [35]:
# to exmple padded inputs
inputs[0,-2:] = 0
inputs = inputs.contiguous()
inputs

tensor([[4, 8, 4, 6, 4, 9, 0, 0],
        [5, 1, 2, 1, 2, 1, 8, 2]])

In [32]:
# all the padding 0 select correspoding value 
# from postion embedding look up table
pos_mask = inputs.eq(0)
position.masked_fill(pos_mask, 0)

tensor([[1, 2, 3, 4, 5, 6, 0, 0],
        [1, 2, 3, 4, 5, 6, 7, 8]])

# 3. Decoder class

In [321]:
class DecoderLayer(nn.Module):
    """
    1st attn layer: self attention layer
    - Q, K, V = decoder inputs
    2nd attn layer: decoder encoder attention layer
    - Q, K, V = 1st attn layer outputs, encoder outputs, encoder outputs
    
    MultiHeadAttention return
    - attn_outputs: [bs, seq_q_len, d_hidn]
    - attn_prob: [bs, n_head, seq_q_len, seq_k_len]
    - dec_enc_attn_prob: [bs, n_head, n_dec_seq, n_enc_seq]
    """
    def __init__(self, config):
        super().__init__()
        self.config = config
        
        self.self_attn = MultiHeadAttention(self.config)
        self.layer_norm1 = nn.LayerNorm(self.config.d_hidn, eps=self.config.layer_norm_epsilon)
        
        self.dec_enc_attn = MultiHeadAttention(self.config)
        self.layer_norm2 = nn.LayerNorm(self.config.d_hidn, eps=self.config.layer_norm_epsilon)
        
        self.pos_feedforward = PoswiseFeedForwardNet(self.config)
        self.layer_norm3 = nn.LayerNorm(self.config.d_hidn, eps=self.config.layer_norm_epsilon)
        
    def forward(self, dec_inputs, enc_outputs, self_attn_mask, dec_enc_attn_mask):
        self_attn_outputs, self_attn_prob = self.self_attn(dec_inputs, dec_inputs, dec_inputs, self_attn_mask) 
        self_attn_outputs = self.layer_norm1(dec_inputs + self_attn_outputs) # LaterNorm(residual)
        
        dec_enc_attn_outputs, dec_enc_attn_prob = self.dec_enc_attn(self_attn_outputs, enc_outputs, enc_outputs, dec_enc_attn_mask)
        dec_enc_attn_outputs = self.layer_norm2(self_attn_outputs + dec_enc_attn_outputs) # LaterNorm(residual)
        
        ffn_outputs = self.pos_feedforward(dec_enc_attn_outputs)
        ffn_outputs = self.layer_norm3(dec_enc_attn_outputs + ffn_outputs)
        
        return ffn_outputs, self_attn_prob, dec_enc_attn_prob

In [361]:
class Decoder(nn.Module):
    """
    variables
    - dec_inputs: [bs, seq_len]
    - dec_outputs: token & postion embedded inputs
        - shape: [bs, seq_len, d_hidn]
    - dec_attn_pad_mask: mask padded key seq
    - dec_attn_decoder_mask: mask triangular part
    - dec_self_attn_mask: dec_attn_pad_mask + dec_attn_decoder_mask
        - or(+) operation both "dec_attn_pad_mask" and "dec_attn_decoder_mask"
    - dec_enc_attn_mask: masking key vector(=enc_inputs)
    
    """
    def __init__(self, config):
        super().__init__()
        self.config = config
        
        self.dec_emb = nn.Embedding(self.config.n_dec_vocab, self.config.d_hidn)
        sinusoid_table = torch.FloatTensor(get_sinusoid_encoding_table(self.config.n_dec_seq + 1, self.config.d_hidn))
        self.pos_emb = nn.Embedding.from_pretrained(sinusoid_table, freeze=True)
        self.layers = nn.ModuleList([DecoderLayer(self.config) for _ in range(self.config.n_layer)])
        
    def forward(self, dec_inputs, enc_inputs, enc_outputs):
        position = torch.arange(dec_inputs.size(1), device=dec_inputs.device, dtype=dec_inputs.dtype)\
                        .expand(dec_inputs.size(0), dec_inputs.size(1)).contiguous() + 1
        pos_mask = dec_inputs.eq(self.config.i_pad)
        position.masked_fill_(pos_mask, 0)
        
        dec_outputs = self.dec_emb(dec_inputs) + self.pos_emb(position)
        
        dec_attn_pad_mask = get_attn_pad_mask(dec_inputs, dec_inputs, self.config.i_pad) # get_attn_pad_mask(seq_q, seq_k, i_pad)
        dec_attn_decoder_mask = get_attn_decoder_mask(dec_inputs) # 대각선 기준 위 mask
        dec_self_attn_mask = torch.gt((dec_attn_pad_mask + dec_attn_decoder_mask), 0)
        
        dec_enc_attn_mask = get_attn_pad_mask(dec_inputs, enc_inputs, self.config.i_pad)
        
        self_attn_probs, dec_enc_attn_probs = [], []
        for layer in self.layers:
            # (bs, n_dec_seq, d_hidn), (bs, n_dec_seq, n_dec_seq), (bs, d_dec_seq, n_enc_seq)
            dec_outputs, self_attn_prob, dec_enc_attn_prob = layer(dec_outputs, enc_outputs, dec_self_attn_mask, dec_enc_attn_mask)
            self_attn_probs.append(self_attn_prob)
            dec_enc_attn_probs.append(dec_enc_attn_prob)
        return dec_outputs, self_attn_probs, dec_enc_attn_probs

# 4. Transformer class

In [310]:
class Transformer(nn.Module):
    """
    variables
    - enc_outputs: [bs, len_enc_seq, d_hidn]
    - enc_self_attn_probs: [bs, n_head, len_enc_seq, len_enc_seq]
    - dec_outputs: [bs, len_seq, d_hidn]
    - dec_self_attn_probs: [bs, n_head, len_dec_seq, len_dec_seq]
    - dec_enc_dec_porbs: [bs, n_head, len_dec_seq, len_enc_seq]
    """
    def __init__(self, config):
        super().__init__()
        self.config = config 
        
        self.encoder = Encoder(self.config)
        self.decoder = Decoder(self.config)
        
    def forward(self, enc_inputs, dec_inputs):
        enc_outputs, enc_self_attn_probs = self.encoder(enc_inputs)
        dec_outputs, dec_self_attn_probs, dec_enc_attn_probs = self.decoder(dec_inputs, enc_inputs, enc_outputs)
        return dec_outputs, enc_self_attn_probs, dec_self_attn_probs, dec_enc_attn_probs

---
# 5. Dataset 
- Naver movie

- dataset class tutorial
    - https://pytorch.org/tutorials/beginner/data_loading_tutorial.html

In [311]:
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

class CustomDataset(Dataset):
    """
    note: decoding inputs have "[BOS]" only, because it predict 1, 0 sentimental label
    self.vocab: sentencepiece object
    self.labels: sentiment lable
        - eg. [1, 0, 0, 0, 0, 1, 0, 0, 0, 1]
    self.sentence: review sentence
        - eg. [ 'GDNTOPCLASSINTHECLUB', '뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아',]
    """
    def __init__(self, vocab, data_fn):
        self.vocab = vocab
        self.labels = []
        self.sentence = []
        
        with open(fn, 'r') as f:
            for i, json_data in enumerate(f):
                if i == 50:
                    break
                json_data = json.loads(json_data)
                self.labels.append(json_data['label'])
                self.sentence.append([vocab.piece_to_id(tok) for tok in json_data['doc']])

    def __len__(self):
        assert len(self.labels) == len(self.sentence)
        return len(self.labels)
    
    def __getitem__(self, idx):
        return (torch.tensor(self.labels[idx]), 
               torch.tensor(self.sentence[idx]),
               torch.tensor([self.vocab.piece_to_id("[BOS]")])) # input only [BOS] into Decoder 

In [347]:
def custom_collate_fn(batch_data):
    """
    batch_data: n data(n=batch size) from Dataset 
    return
    - label
    - enc_inputs(token_ids): [bs, seq_len]
    - dec_inputs(BOS_token): [bs, seq_len]
    """
    label, token_ids, BOS_token = list(zip(*batch_data))
    
    token_ids = nn.utils.rnn.pad_sequence(token_ids, batch_first=True, padding_value=0)
    BOS_token = nn.utils.rnn.pad_sequence(BOS_token, batch_first=True, padding_value=0)
    
    return [torch.stack(label, dim=0), token_ids, BOS_token] # [label, encoder inputs, decoder inputs]

In [348]:
bs = 128
train_dataset = CustomDataset(vocab, data_fn='ratings_train.json')
train_loader = DataLoader(train_dataset, batch_size=bs, collate_fn=custom_collate_fn)
test_dataset = CustomDataset(vocab, data_fn='ratings_test.json')
test_loader = DataLoader(test_dataset, batch_size=bs, collate_fn=custom_collate_fn)

### generate dataset example code

- preprocess text data

In [73]:
labels = []; sentence = []; d = []; line_cnt = 0; data = [];
fn = 'ratings_test.txt'
with open(fn) as f:
    while True:
        line = f.readline()
        line_cnt += 1
        if line_cnt == 1:
            continue
        if not line:
            break
        id_, sent, label = line.split('\t')
        label = label.split('\n')[0]
        labels.append(int(label))
        sentence.append(sent)

In [77]:
labels[:10], sentence[:10]

([1, 0, 0, 0, 0, 1, 0, 0, 0, 1],
 ['굳 ㅋ',
  'GDNTOPCLASSINTHECLUB',
  '뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아',
  '지루하지는 않은데 완전 막장임... 돈주고 보기에는....',
  '3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??',
  '음악이 주가 된, 최고의 음악영화',
  '진정한 쓰레기',
  '마치 미국애니에서 튀어나온듯한 창의력없는 로봇디자인부터가,고개를 젖게한다',
  '갈수록 개판되가는 중국영화 유치하고 내용없음 폼잡다 끝남 말도안되는 무기에 유치한cg남무 아 그립다 동사서독같은 영화가 이건 3류아류작이다',
  '이별의 아픔뒤에 찾아오는 새로운 인연의 기쁨 But, 모든 사람이 그렇지는 않네..'])

- convert preprocess data to json

In [76]:
target_data = {'id': id_, 'doc': vocab.encode_as_pieces(sent), 'label':label}
with open('ratings_train.json', 'w') as f:
    f.write(json.dumps(target_data))
    f.write('\n')

- load json data

In [91]:
fn = '../ratings_train.json'
with open(fn, 'r') as f:
    iterator = iter(f)
    sample_data = next(iterator) # iter json data

data = json.loads(sample_data)
print('json data:', data, '\ntok_to_id:', [vocab.piece_to_id(tok) for tok in data['doc']])

json data: {'id': 9976970, 'doc': ['▁아', '▁더', '빙', '..', '▁진', '짜', '▁', '짜', '증', '나', '네', '요', '▁목', '소', '리'], 'label': 0} 
tok_to_id: [26, 228, 4365, 1920, 132, 4351, 3587, 4351, 3922, 3628, 3857, 3760, 266, 3678, 3614]


# 6. clf model

In [364]:
class ReviewSentClf(nn.Module):
    """
    Transformer classification
    - average or max pooling over hidden_dim -> linear(in_feature=-1, out_feature=n_class)
    
    torch.max() return values and indices
    - dec_outputs(values): [bs, d_hidn] <- max pooling
    
    logits: [bs, n_output]
    """
    def __init__(self, config):
        super().__init__()
        self.config = config
        self.transformer = Transformer(self.config)
        self.feedforward = nn.Linear(self.config.d_hidn, self.config.n_output, bias=False)
        
    def forward(self, enc_inputs, dec_inputs):
        dec_outputs, enc_self_attn_probs, dec_self_attn_probs, dec_enc_attn_probs = self.transformer(enc_inputs, dec_inputs)
        dec_outputs, _ = torch.max(dec_outputs, dim=1)
        logits = self.feedforward(dec_outputs)
        
        return logits, enc_self_attn_probs, dec_self_attn_probs, dec_enc_attn_probs

# 7. train code

In [377]:
def eval_model(config, model, data_loader):
    matchs = []
    model.eval()
    
    n_word_total = 0
    with torch.no_grad():
        for i, batch in enumerate(data_loader):
            labels, enc_inputs, dec_inputs = map(lambda x: x.to(config.device), batch)
            
            outputs = model(enc_inputs, dec_inputs)
            logits = outputs[0] # logist = feedforard(dec_outputs): [bs, n_out]
            val, idx = logits.max(axis=1) # eg. binary clf: [3, 5] -> idx: [1]; idx shape: [bs]
            
            match = torch.eq(idx, labels).detach().cpu()
            matchs.extend(match.detach().cpu())
            
            running_accuracy = np.sum(matchs) / len(matchs) if 0 < len(matchs) else 0
            print(running_accuracy)
            
    return running_accuracy

In [378]:
def train_model(config, n_epoch, model, criterion, optimizer, data_loader):
    model.train()
    running_loss = 0
    for i, batch in enumerate(data_loader):
        labels, enc_inputs, dec_inputs = map(lambda x: x.to(config.device), batch)
        optimizer.zero_grad()
        outputs = model(enc_inputs, dec_inputs)
        logits = outputs[0]
        
        loss = criterion(logits, labels)
        loss_val = loss.item()
        running_loss += loss_val
        
        loss.backward()
        optimizer.step()
        
        print('batch loss average: {}'.format(running_loss/(i+1)))
    
    return running_loss / len(data_loader)

In [325]:
config.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
config.n_output = 2

learning_rate = 5e-5
n_epoch = 10

In [376]:
model = ReviewSentClf(config)
model.to(config.device)

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

loss_ls, score_ls = [], []
best_epoch, best_loss, best_score = 0, 0, 0
for epoch in range(n_epoch):
    loss = train_model(config, epoch, model, criterion, optimizer, train_loader)
    score = eval_model(config, model, test_loader)
    loss_ls.append(loss)
    score_ls.append(score)
    
    if best_score < score:
        best_epoch, best_loss, best_score = epoch, loss, score

batch loss average: 0.7985257506370544
0.44


KeyboardInterrupt: 